# Pré-processamento dos dados iMIGUE

Notebook para pré-processamento dos dados iMIGUE, incluindo leitura, normalização, ajuste de tamanho, divisão e salvamento dos dados.

## 1. Importação de Bibliotecas
Importe os pacotes necessários: os, numpy, pandas, sklearn.model_selection.train_test_split.

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 2. Configuração de Diretórios e Parâmetros
Defina os diretórios de entrada/saída, arquivo de labels e parâmetros como N_FRAMES.

In [2]:
# Diretórios e parâmetros
SKELETON_DIR = "../../data/iMIGUE/mg_skeleton_only"   # pasta onde estão os .xlsx
LABELS_FILE = "../../data/iMIGUE/Label/labels_20200831.csv"       # CSV com mapeamento vídeo -> label
OUTPUT_DIR = "../../data/processed_data"    # onde salvar os npy
N_FRAMES = 300                   # número fixo de frames por vídeo

os.makedirs(OUTPUT_DIR, exist_ok=True)

## 3. Função de Padding/Truncagem de Sequências
Implemente a função pad_or_truncate para ajustar sequências ao tamanho fixo.

In [3]:
def pad_or_truncate(sequence, target_len=N_FRAMES):
    """
    Ajusta a sequência para um comprimento fixo (padding com zeros ou truncagem).
    """
    n_frames, n_features = sequence.shape
    if n_frames > target_len:
        return sequence[:target_len, :]
    elif n_frames < target_len:
        padding = np.zeros((target_len - n_frames, n_features))
        return np.vstack([sequence, padding])
    return sequence

## 4. Carregamento e Mapeamento dos Labels
Carregue o arquivo CSV de labels, crie o mapeamento vídeo->classe e codifique as classes em inteiros.

In [4]:
# Carregar labels
labels_df = pd.read_csv(LABELS_FILE)
label_map = dict(zip(labels_df["video_id"].astype(str).apply(lambda x: x.split('_')[0].zfill(4)), labels_df["class"]))

# Codificação das classes
unique_labels = sorted(labels_df["class"].unique())
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}

## 5. Busca de Vídeos RGB Existentes
Percorra os diretórios RGB e obtenha os IDs dos vídeos disponíveis.

In [5]:
RGB_DIRS = [
    "../../data/iMIGUE/iMiGUE_RGB_Phase1/imigue_rgb_train",
    "../../data/iMIGUE/iMiGUE_RGB_Phase1/imigue_rgb_validate",
    "../../data/iMIGUE/iMiGUE_RGB_Phase2/imigue_rgb_test"
 ]
rgb_videos = set()
for rgb_dir in RGB_DIRS:
    if os.path.exists(rgb_dir):
        for folder_name in os.listdir(rgb_dir):
            folder_path = os.path.join(rgb_dir, folder_name)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path):
                    if file == '.DS_Store':
                        continue
                    ext = os.path.splitext(file)[1].lower()
                    if ext in ['.mp4', '.avi', '.mov']:
                        video_id_rgb = os.path.splitext(file)[0]  # preserva zeros à esquerda
                        # Só adiciona se o nome da pasta for igual ao nome do vídeo
                        if folder_name == video_id_rgb:
                            rgb_videos.add(video_id_rgb)
                            print(f'RGB encontrado: pasta={folder_name}, arquivo={file}, id={video_id_rgb}')

RGB encontrado: pasta=0184, arquivo=0184.mp4, id=0184
RGB encontrado: pasta=0342, arquivo=0342.mp4, id=0342
RGB encontrado: pasta=0170, arquivo=0170.mp4, id=0170
RGB encontrado: pasta=0183, arquivo=0183.mp4, id=0183
RGB encontrado: pasta=0141, arquivo=0141.mp4, id=0141
RGB encontrado: pasta=0179, arquivo=0179.mp4, id=0179
RGB encontrado: pasta=0374, arquivo=0374.mp4, id=0374
RGB encontrado: pasta=0115, arquivo=0115.mp4, id=0115
RGB encontrado: pasta=0311, arquivo=0311.mp4, id=0311
RGB encontrado: pasta=0124, arquivo=0124.mp4, id=0124
RGB encontrado: pasta=0316, arquivo=0316.mp4, id=0316
RGB encontrado: pasta=0178, arquivo=0178.mp4, id=0178
RGB encontrado: pasta=0147, arquivo=0147.mp4, id=0147
RGB encontrado: pasta=0381, arquivo=0381.mp4, id=0381
RGB encontrado: pasta=0386, arquivo=0386.mp4, id=0386
RGB encontrado: pasta=0140, arquivo=0140.mp4, id=0140
RGB encontrado: pasta=0149, arquivo=0149.mp4, id=0149
RGB encontrado: pasta=0344, arquivo=0344.mp4, id=0344
RGB encontrado: pasta=0176, 

In [6]:
# Depuração: listar diretórios RGB e subpastas
for rgb_dir in RGB_DIRS:
    print(f'RGB_DIR: {rgb_dir}')
    if os.path.exists(rgb_dir):
        subfolders = [f for f in os.listdir(rgb_dir) if os.path.isdir(os.path.join(rgb_dir, f))]
        print(f'  Subpastas encontradas: {subfolders}')
        for folder_name in subfolders:
            folder_path = os.path.join(rgb_dir, folder_name)
            files = os.listdir(folder_path)
            print(f'    Arquivos em {folder_name}: {files}')

RGB_DIR: ../../data/iMIGUE/iMiGUE_RGB_Phase1/imigue_rgb_train
  Subpastas encontradas: ['0184', '0342', '0170', '0183', '0141', '0179', '0374', '0115', '0311', '0124', '0316', '0178', '0147', '0381', '0386', '0140', '0149', '0344', '0176', '0171', '0343', '0185', '0317', '0328', '0122', '0310', '0319', '0113', '0298', '0095', '0253', '0061', '0405', '0059', '0402', '0066', '0254', '0050', '0296', '0068', '0291', '0433', '0057', '0231', '0236', '0032', '0035', '0056', '0264', '0290', '0297', '0263', '0051', '0435', '0058', '0093', '0403', '0299', '0060', '0252', '0404', '0094', '0239', '0230', '0018', '0027', '0223', '0029', '0016', '0270', '0419', '0089', '0283', '0045', '0277', '0248', '0087', '0073', '0246', '0279', '0028', '0225', '0017', '0222', '0019', '0026', '0214', '0075', '0247', '0240', '0086', '0276', '0420', '0282', '0249', '0043', '0131', '0304', '0136', '0332', '0100', '0107', '0335', '0153', '0392', '0366', '0359', '0162', '0350', '0357', '0165', '0191', '0334', '0101', 

## 6. Processamento dos Arquivos de Skeleton
Para cada arquivo .xlsx, carregue os dados, normalize, ajuste o tamanho e adicione ao dataset se o vídeo existir nos labels e RGB.

In [7]:
import time
X, y = [], []
skeleton_ids = set()
xlsx_files = []
for root, dirs, files in os.walk(SKELETON_DIR):
    for file in files:
        if file.endswith(".xlsx"):
            xlsx_files.append((root, file))
n_total = len(xlsx_files)
print(f"Processando {n_total} arquivos de skeleton...")
start_time = time.time()
for idx, (root, file) in enumerate(xlsx_files):
    video_id_full = os.path.splitext(file)[0]  # ex.: "0001_2"
    video_id = video_id_full.split('_')[0]  # preserva zeros à esquerda
    skeleton_ids.add(video_id)
    
    if video_id not in label_map:
        continue
    if video_id not in rgb_videos:
        continue
    # Carregar skeleton
    df = pd.read_excel(os.path.join(root, file))
    seq = df.values.astype(np.float32)  # (n_frames, 411)
    
    # Normalizar coordenadas (0-1)
    seq = np.nan_to_num(seq)  # substitui NaNs por 0
    
    # Ajustar tamanho
    seq_fixed = pad_or_truncate(seq, target_len=N_FRAMES)
    
    # Verificar shape antes de adicionar
    if seq_fixed.shape != (N_FRAMES, 411):
        print(f"Arquivo ignorado: {file} - shape encontrado: {seq_fixed.shape}")
        continue
    # Adicionar ao dataset apenas se válido
    X.append(seq_fixed)
    y.append(label_to_int[label_map[video_id]])
    
    # Print progresso a cada 5%
    if n_total > 0 and idx % max(1, n_total // 20) == 0:
        percent = int(100 * (idx + 1) / n_total)
        elapsed = time.time() - start_time
        print(f"Progresso: {percent}% ({idx+1}/{n_total}) - {elapsed:.1f}s")
X = np.array(X)  # (n_videos, N_FRAMES, 411)
y = np.array(y)

Processando 359 arquivos de skeleton...
Progresso: 0% (1/359) - 9.7s
Progresso: 0% (1/359) - 9.7s
Progresso: 5% (18/359) - 315.3s
Progresso: 5% (18/359) - 315.3s
Progresso: 9% (35/359) - 568.4s
Progresso: 9% (35/359) - 568.4s
Progresso: 14% (52/359) - 939.0s
Progresso: 14% (52/359) - 939.0s
Progresso: 19% (69/359) - 1284.3s
Progresso: 19% (69/359) - 1284.3s
Progresso: 23% (86/359) - 1506.2s
Progresso: 23% (86/359) - 1506.2s
Progresso: 28% (103/359) - 1746.5s
Progresso: 28% (103/359) - 1746.5s
Progresso: 33% (120/359) - 2032.0s
Progresso: 33% (120/359) - 2032.0s
Progresso: 38% (137/359) - 2355.9s
Progresso: 38% (137/359) - 2355.9s
Progresso: 42% (154/359) - 2672.1s
Progresso: 42% (154/359) - 2672.1s
Progresso: 47% (171/359) - 3108.6s
Progresso: 47% (171/359) - 3108.6s
Progresso: 52% (188/359) - 3653.7s
Progresso: 52% (188/359) - 3653.7s
Arquivo ignorado: 0348_2.xlsx - shape encontrado: (300, 0)
Arquivo ignorado: 0348_2.xlsx - shape encontrado: (300, 0)
Progresso: 57% (205/359) - 4495.0s

## 7. Divisão dos Dados em Treino, Validação e Teste
Utilize train_test_split para dividir X e y em conjuntos de treino, validação e teste, mantendo a proporção das classes.

In [8]:
# Depuração: verificar shapes dos elementos de X
shapes = [x.shape for x in X]
print("Shapes únicos encontrados em X:", set(shapes))
if all(s == (N_FRAMES, 411) for s in shapes):
    X = np.array(X)
else:
    print("Atenção: Existem elementos em X com shape diferente de (300, 411). Não foi possível converter para array.")
if isinstance(y, list):
    y = np.array(y)
print("Shape de X:", getattr(X, 'shape', 'lista'))
print("Shape de y:", getattr(y, 'shape', 'lista'))
print("Quantidade de skeletons processados:", len(X))

Shapes únicos encontrados em X: {(300, 411)}
Shape de X: (357, 300, 411)
Shape de y: (357,)
Quantidade de skeletons processados: 357


In [9]:
# Depuração: listar arquivos de skeleton encontrados
print('Skeleton IDs encontrados:', sorted(list(skeleton_ids)))

# Depuração: listar IDs de vídeos RGB encontrados
print('RGB video IDs encontrados:', sorted(list(rgb_videos)))

# Depuração: listar IDs de labels encontrados
print('Label IDs encontrados:', sorted(list(label_map.keys())))

Skeleton IDs encontrados: ['0001', '0002', '0003', '0004', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0022', '0023', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0032', '0035', '0036', '0037', '0038', '0039', '0040', '0043', '0045', '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '0060', '0061', '0062', '0063', '0064', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0079', '0082', '0083', '0084', '0085', '0086', '0087', '0089', '0091', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0108', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0118', '0119', '0120', '0121', '0122', '0124', '0125', '0126', '0131', '0132', '0133', '0134', '0135', '0136', '0137', '0138', '0140', '0141', '0143', '0144

In [11]:
# Filtrar classes com apenas 1 amostra para evitar erro no split estratificado
import numpy as np
from collections import Counter
class_counts = Counter(y)
classes_validas = [cls for cls, count in class_counts.items() if count > 1]
mask = np.isin(y, classes_validas)
X_filtrado = X[mask]
y_filtrado = y[mask]
print(f"Classes removidas (apenas 1 amostra): {[cls for cls, count in class_counts.items() if count == 1]}")
print(f"Novo shape de X: {X_filtrado.shape}")
print(f"Novo shape de y: {y_filtrado.shape}")

Classes removidas (apenas 1 amostra): [np.int64(15), np.int64(14), np.int64(0), np.int64(8), np.int64(6)]
Novo shape de X: (352, 300, 411)
Novo shape de y: (352,)


In [15]:
# Filtrar classes com apenas 1 amostra em y_temp antes do segundo split
from collections import Counter
class_counts_temp = Counter(y_temp)
classes_validas_temp = [cls for cls, count in class_counts_temp.items() if count > 1]
mask_temp = np.isin(y_temp, classes_validas_temp)
X_temp_filtrado = X_temp[mask_temp]
y_temp_filtrado = y_temp[mask_temp]
print(f"Classes removidas do temp (apenas 1 amostra): {[cls for cls, count in class_counts_temp.items() if count == 1]}")
print(f"Novo shape de X_temp: {X_temp_filtrado.shape}")
print(f"Novo shape de y_temp: {y_temp_filtrado.shape}")

# Segundo split estratificado
X_val, X_test, y_val, y_test = train_test_split(
    X_temp_filtrado, y_temp_filtrado, test_size=0.50, stratify=y_temp_filtrado, random_state=42
)
print(f"Shapes: X_val={X_val.shape}, X_test={X_test.shape}")
print(f"Shapes: y_val={y_val.shape}, y_test={y_test.shape}")

Classes removidas do temp (apenas 1 amostra): [np.int64(12), np.int64(3), np.int64(27), np.int64(11), np.int64(20), np.int64(10), np.int64(13), np.int64(16)]
Novo shape de X_temp: (98, 300, 411)
Novo shape de y_temp: (98,)
Shapes: X_val=(49, 300, 411), X_test=(49, 300, 411)
Shapes: y_val=(49,), y_test=(49,)


## 8. Salvamento dos Dados em Arquivos .npy
Salve os arrays X_train, y_train, X_val, y_val, X_test, y_test em arquivos .npy no diretório de saída.

In [17]:
np.save(os.path.join(OUTPUT_DIR, "X_train.npy"), X_train)
np.save(os.path.join(OUTPUT_DIR, "y_train.npy"), y_train)
np.save(os.path.join(OUTPUT_DIR, "X_val.npy"), X_val)
np.save(os.path.join(OUTPUT_DIR, "y_val.npy"), y_val)
np.save(os.path.join(OUTPUT_DIR, "X_test.npy"), X_test)
np.save(os.path.join(OUTPUT_DIR, "y_test.npy"), y_test)

print("✅ Dados salvos em", OUTPUT_DIR)

✅ Dados salvos em ../../data/processed_data


In [18]:
# Salvar os labels dos splits em CSV para análise
pd.DataFrame(y_train, columns=["label"]).to_csv(os.path.join(OUTPUT_DIR, "y_train.csv"), index=False)
pd.DataFrame(y_val, columns=["label"]).to_csv(os.path.join(OUTPUT_DIR, "y_val.csv"), index=False)
pd.DataFrame(y_test, columns=["label"]).to_csv(os.path.join(OUTPUT_DIR, "y_test.csv"), index=False)
print("✅ Labels salvos em CSV no diretório de saída.")

✅ Labels salvos em CSV no diretório de saída.


In [20]:
import numpy as np

# Carregar arquivos
X_train = np.load("../../data/processed_data/X_train.npy")
y_train = np.load("../../data/processed_data/y_train.npy")
X_val = np.load("../../data/processed_data/X_val.npy")
y_val = np.load("../../data/processed_data/y_val.npy")
X_test = np.load("../../data/processed_data/X_test.npy")
y_test = np.load("../../data/processed_data/y_test.npy")

# Conferir shapes
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

X_train: (246, 300, 411) y_train: (246,)
X_val: (49, 300, 411) y_val: (49,)
X_test: (49, 300, 411) y_test: (49,)
